In [ ]:
import pandas as pd

X = pd.read_csv('X_updated.csv')
y = pd.read_csv('y_updated.csv')

In [ ]:
y = y.drop(columns='Unnamed: 0')
X = X.drop(columns='Unnamed: 0')

In [ ]:
X['ProductItem_Combined'] = y 

In [ ]:
mode= X.Business_service.mode()

In [ ]:
X['Business_service'] = X.Business_service.fillna(mode[0])

In [ ]:
# Group the data by the class column
groupedModel_number = X.groupby('Business_service')

# Find the size of each group
group_sizesModel_number = groupedModel_number.size()

# Find the class names of the smallest groups
to_dropModel_number = group_sizesModel_number[group_sizesModel_number  < 2].index

In [ ]:
X = X[~X['Business_service'].isin(to_dropModel_number)]

In [ ]:
y = pd.DataFrame(X['ProductItem_Combined'])
X= X.drop(columns='ProductItem_Combined')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split( X, y, test_size=0.33, random_state=15000, stratify= X.Business_service)
tempSimTrain= pd.DataFrame()
tempSimTest = pd.DataFrame()
tempSimTrain['avg_sim'] = X_train['name_similarity']
tempSimTest['avg_sim'] = X_test['name_similarity']

In [ ]:
pred = ['Company','Model_ID','Model_number','Business_service']

In [ ]:
X_train[pred].describe()

In [ ]:
X_test[pred].describe()

In [ ]:
from sklearn.preprocessing import OneHotEncoder as SklearnOneHotEncoder

class OneHotEncoder(SklearnOneHotEncoder):
    def __init__(self, **kwargs):
        super(OneHotEncoder, self).__init__(**kwargs)
        self.fit_flag = False

    def fit(self, X, **kwargs):
        out = super().fit(X)
        self.fit_flag = True
        return out

    def transform(self, X, **kwargs):
        sparse_matrix = super(OneHotEncoder, self).transform(X)
        new_columns = self.get_new_columns(X=X)
        d_out = pd.DataFrame(sparse_matrix.toarray(), columns=new_columns, index=X.index)
        return d_out

    def fit_transform(self, X, **kwargs):
        self.fit(X)
        return self.transform(X)

    def get_new_columns(self, X):
        new_columns = []
        for i, column in enumerate(X.columns):
            j = 0
            while j < len(self.categories_[i]):
                new_columns.append(f'{column}_<{self.categories_[i][j]}>')
                j += 1
        return new_columns

In [ ]:
X.iloc[:, [0,1,2,3,4]]

In [ ]:
import numpy as np
encoderX = OneHotEncoder()
encoderY =  OneHotEncoder()

X_train = encoderX.fit_transform(X_train.iloc[:, [0,1,2,3,4]])
X_test = encoderX.fit_transform(X_test.iloc[:, [0,1,2,3,4]])
y_train= encoderY.fit_transform(y_train)
y_test = encoderY.fit_transform(y_test)

X_train['name_similarity'] = tempSimTrain['avg_sim']
X_test['name_similarity']  =  tempSimTest['avg_sim']

In [ ]:
X_test

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
 # Imports
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2
import keras
# Create the model
def build_MLP():
    model_ann = Sequential()
    model_ann.add(Dense(480, activation='relu', input_dim=818))
    model_ann.add(Dropout(0.5))
    model_ann.add(Dense(240, activation='relu'))
    model_ann.add(Dropout(0.5))
    model_ann.add(Dense(17, activation='softmax'))
    model_ann.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=0.001, decay=1e-6, clipvalue=0.5), metrics=['Recall','Precision', 'accuracy'])
    return model_ann

In [ ]:
model_ann = build_MLP()

In [ ]:
historyANN = model_ann.fit(X_train, y_train, epochs=10, batch_size= 100)
#validation_data=(X_test, y_test)

In [ ]:
model_ann.evaluate(X_test, y_test )

In [ ]:
from keras.utils.vis_utils import plot_model

In [ ]:
from keras.callbacks import TensorBoard

tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1)

model_ann.fit(X_train, y_train, epochs=10, callbacks=[tensorboard_callback])


In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
model_ann.evaluate(X_test, y_test)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from keras.utils import to_categorical
import numpy as np

# Generate predictions on the test set
y_pred = model_ann.predict(X_test)

pred_classes = np.argmax(y_pred, axis=1)
true_classes = np.argmax(np.array(y_test), axis=1)


conf_matrix = confusion_matrix(true_classes, pred_classes, labels=range(17))

plt.figure(figsize =(10, 5)) 
sns.heatmap(conf_matrix, annot = True, fmt ="d")

In [ ]:
classess = []
for i in range(0,17):
    classess.append(f"Class {i}")
    
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1_score, support = precision_recall_fscore_support(true_classes, pred_classes)

for i, c in enumerate(classess):
    print(f"Class {c} - Precision: {precision[i]:.2f}, Recall: {recall[i]:.2f}, F1-Score: {f1_score[i]:.2f}, Support: {support[i]}")

In [ ]:
import matplotlib.pyplot as plt

# Extract the training and validation accuracy
acc = historyANN.history['accuracy']
val_acc = historyANN.history['val_accuracy']
val_prec = historyANN.history['val_precision']
prec = historyANN.history['precision']
val_recall = historyANN.history['val_recall']
recall = historyANN.history['recall']

# Plot the accuracy
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Accuracy of Best Model')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
X

In [ ]:
frame = pd.read_csv('frame_updated_test.csv')

In [ ]:
# Group the data by the class column
groupedModel_number = frame.groupby('Business_service')

# Find the size of each group
group_sizesModel_number = groupedModel_number.size()

# Find the class names of the smallest groups
to_dropModel_number = group_sizesModel_number[group_sizesModel_number  < 2].index
frame = frame[~frame['Business_service'].isin(to_dropModel_number)]

In [ ]:
frame.Business_service.value_counts()

# Prediction with ANN

In [ ]:
label_mapping_ann = {index: label for index, label in enumerate(encoderY.get_feature_names_out())}
integer_labels_ann = np.argmax(y_pred, axis=1)
original_labels_test_ann = [label_mapping_ann[x] for x in integer_labels_ann]
original_labels_test_ann = pd.DataFrame(original_labels_test_ann)
original_labels_test_ann['original_index'] = y_test.index
original_labels_test_ann['pred_ANN'] = original_labels_test_ann[0]
original_labels_test_ann = original_labels_test_ann.drop(columns=0)

In [ ]:
#prediction on train set 
#label mapping for training predictions
y_pred_train = model_ann.predict(X_train)
label_mapping_train = {index: label for index, label in enumerate(encoderY.get_feature_names_out())}
integer_labels_train = np.argmax(y_pred_train, axis=1)
original_labels_train = [label_mapping_train[x] for x in integer_labels_train]
original_labels_train = pd.DataFrame(original_labels_train)
original_labels_train['original_index'] = y_train.index
original_labels_train['pred_ANN'] = original_labels_train[0]
original_labels_train = original_labels_train.drop(columns=0)

In [ ]:
joined_set_ann = original_labels_train.merge(original_labels_test_ann, on='original_index', how='outer')
joined_set_ann['pred_ANN'] = joined_set_ann['pred_ANN_x'].combine_first(joined_set_ann['pred_ANN_y'])
joined_set_ann.drop(['pred_ANN_x', 'pred_ANN_y'], axis=1, inplace=True)
joined_set_ann

In [ ]:
frame = frame.reset_index()

In [ ]:
frame

In [ ]:
frame.rename(columns={'index' : 'original_index'}, inplace=True)
Original_frame = frame.merge(joined_set_ann, on= 'original_index', how='outer')
Original_frame.info()

In [ ]:
Original_frame

# CNN model

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input
from keras.optimizers import Adam



import keras
# define the model
def build_cnn ():
    modelCNN = Sequential()
    modelCNN.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(818, 1)))
    modelCNN.add(MaxPooling1D(pool_size=2))
    modelCNN.add(Dropout(.5))
    modelCNN.add(Flatten())
    modelCNN.add(Dense(128, activation='relu'))
    modelCNN.add(Dropout(.2))
    modelCNN.add(Dense(17, activation='softmax'))
    # compile the model
    modelCNN.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=0.01, decay=1e-6, clipvalue=0.5), metrics=['Recall','Precision', 'accuracy'])
    return modelCNN

# tensorboard

In [ ]:
from keras.callbacks import TensorBoard
import numpy as np

#tensorboard_callback_cnn = TensorBoard(log_dir='./logsCNNdipl5', histogram_freq=1)

modelCNN = build_cnn()
#modelCNN.fit(np.array(X_train), np.array(y_train), epochs=10 )

In [ ]:
%tensorboard --logdir logsCNNdipl/fit

In [ ]:
historyCNN = modelCNN.fit(X_train, y_train, epochs=10, validation_data=(X_test,y_test))

In [ ]:
modelCNN.evaluate(X_train, y_train)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from keras.utils import to_categorical
import numpy as np

# Generate predictions on the test set
y_pred_cnn = modelCNN.predict(X_test)

pred_classes = np.argmax(y_pred_cnn, axis=1)
true_classes = np.argmax(np.array(y_test), axis=1)


conf_matrix = confusion_matrix(true_classes, pred_classes, labels=range(17))

plt.figure(figsize =(10, 5)) 
sns.heatmap(conf_matrix, annot = True, fmt ="d")

In [ ]:
classess = []
for i in range(0,17):
    classess.append(f"Class {i}")
    
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1_score, support = precision_recall_fscore_support(true_classes, pred_classes)

for i, c in enumerate(classess):
    print(f"Class {c} - Precision: {precision[i]:.2f}, Recall: {recall[i]:.2f}, F1-Score: {f1_score[i]:.2f}, Support: {support[i]}")

In [ ]:
import matplotlib.pyplot as plt

# Extract the training and validation accuracy
acc = historyCNN.history['accuracy']
val_acc = historyCNN.history['val_accuracy']
val_prec = historyCNN.history['val_precision']
prec = historyCNN.history['precision']
val_recall = historyCNN.history['val_recall']
recall = historyCNN.history['recall']

# Plot the accuracy
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Accuracy of CNN Model')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
np.argmax(y_pred_cnn, axis=1)

In [ ]:
label_mapping_cnn = {index: label for index, label in enumerate(encoderY.get_feature_names_out())}
integer_labels_cnn = np.argmax(y_pred_cnn, axis=1)
original_labels_test_cnn = [label_mapping_cnn[x] for x in integer_labels_cnn]
original_labels_test_cnn = pd.DataFrame(original_labels_test_cnn)
original_labels_test_cnn['original_index'] = y_test.index
original_labels_test_cnn['pred_CNN'] = original_labels_test_cnn[0]
original_labels_test_cnn = original_labels_test_cnn.drop(columns=0)

In [ ]:
#prediction on train set 
#label mapping for training predictions
y_pred_train_Cnn = modelCNN.predict(X_train)
label_mapping_train_cnn = {index: label for index, label in enumerate(encoderY.get_feature_names_out())}
integer_labels_train_cnn = np.argmax(y_pred_train, axis=1)
original_labels_train_cnn = [label_mapping_train_cnn[x] for x in integer_labels_train_cnn]
original_labels_train_cnn = pd.DataFrame(original_labels_train_cnn)
original_labels_train_cnn['original_index'] = y_train.index
original_labels_train_cnn['pred_CNN'] = original_labels_train_cnn[0]
original_labels_train_cnn = original_labels_train_cnn.drop(columns=0)

In [ ]:
label_mapping = {index: label for index, label in enumerate(encoderY.get_feature_names_out())}
integer_labels = np.argmax(y_pred_cnn, axis=1)

def return_original_labels(label_mapping, integer_labels, y):
    original_labels = [label_mapping[x] for x in integer_labels]
    original_labels_test = pd.DataFrame(original_labels)
    original_labels_test['original_index'] = y.index
    original_labels_test['pred_cnn'] = original_labels[0]
    original_labels_test = original_labels_test.drop(columns=0)
    return original_labels_test

original_labels_test= return_original_labels(label_mapping, integer_labels, y_test)
original_labels_test

In [ ]:
#label mapping for training predictions
y_pred_train_cnn = modelCNN.predict(X_train)
label_mapping_train = {index: label for index, label in enumerate(encoderY.get_feature_names_out())}
integer_labels_train = np.argmax(y_pred_train_cnn, axis=1)

# original_labels_train = [label_mapping_train[x] for x in integer_labels_train]
# original_labels_train = pd.DataFrame(original_labels_train)
# original_labels_train['original_index'] = y_train.index
# original_labels_train['pred_cnn'] = original_labels_train[0]
# original_labels_train = original_labels_train.drop(columns=0)

original_labels_train = return_original_labels(label_mapping_train, integer_labels_train, y_train)
original_labels_train

# predictions for CNN 

In [ ]:
joined_set_cnn = original_labels_train_cnn.merge(original_labels_test_cnn, on='original_index', how='outer')
joined_set_cnn['pred_CNN'] = joined_set_cnn['pred_CNN_x'].combine_first(joined_set_cnn['pred_CNN_y'])
joined_set_cnn.drop(['pred_CNN_x', 'pred_CNN_y'], axis=1, inplace=True)
joined_set_cnn

In [ ]:
joined_set_cnn

In [ ]:
test  = Original_frame.merge(X.reset_index().rename(columns={'index' :'original_index'}), on= 'original_index' ) 

In [ ]:
X

In [ ]:

Original_frame = Original_frame.merge(joined_set_cnn, on= 'original_index', how='outer')
Original_frame

In [ ]:
len(Original_frame.ProductItem_Combined.unique())

# Tune hyperpamateres for ANN

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner import RandomSearch


# Define the model-building function
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_1', min_value=32, max_value=1024, step=32), activation='relu', input_dim=818))
    model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Dense(units=hp.Int('units_2', min_value=32, max_value=1024, step=32), activation='relu'))
    model.add(Dropout(hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Dense(17, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(lr=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), metrics=['Recall', 'Precision', 'accuracy'])
    return model

# Instantiate the tuner and perform the hyperparameter search
tuner = RandomSearch(build_model,objective='val_accuracy', max_trials=10, executions_per_trial=2,directory='my_dir2', project_name='updated_set_ANN1_to_join')

tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Print the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: {best_hps}")

# Build the model with the best hyperparameters and train it
model = tuner.hypermodel.build(best_hps)
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

In [ ]:
import matplotlib.pyplot as plt

# Extract the training and validation accuracy
acc = model.history.history['accuracy']
val_acc = model.history.history['val_accuracy']
val_prec = model.history.history['val_precision']
prec = model.history.history['precision']
val_recall = model.history.history['val_recall']
recall = model.history.history['recall']

# Plot the accuracy
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Accuracy of Best Model')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from keras.utils import to_categorical
import numpy as np

# Generate predictions on the test set
y_bm_pred_test = model.predict(X_test)

pred_classes = np.argmax(y_bm_pred_test, axis=1)
true_classes = np.argmax(np.array(y_test), axis=1)


conf_matrix = confusion_matrix(true_classes, pred_classes, labels=range(17))

plt.figure(figsize =(10, 5)) 
sns.heatmap(conf_matrix, annot = True, fmt ="d")

In [ ]:
y_bm_pred_test = model.predict(X_test)

# predict and map results from BM ANN

In [ ]:
label_mapping_ann_BM = {index: label for index, label in enumerate(encoderY.get_feature_names_out())}
integer_labels_ann_BM = np.argmax(y_bm_pred_test, axis=1)
original_labels_test_ann_BM = [label_mapping_ann_BM[x] for x in integer_labels_ann_BM]
original_labels_test_ann_BM = pd.DataFrame(original_labels_test_ann_BM)
original_labels_test_ann_BM['original_index'] = y_test.index
original_labels_test_ann_BM['pred_ANN_BM'] = original_labels_test_ann_BM[0]
original_labels_test_ann_BM = original_labels_test_ann_BM.drop(columns=0)


In [ ]:
#prediction on train set 
#label mapping for training predictions
y_pred_train_ann_BM = model.predict(X_train)
label_mapping_train_BM = {index: label for index, label in enumerate(encoderY.get_feature_names_out())}
integer_labels_train_ann_BM = np.argmax(y_pred_train_ann_BM, axis=1)
original_labels_train_ann_bm = [label_mapping_train_BM[x] for x in integer_labels_train_ann_BM]
original_labels_train_ann_bm = pd.DataFrame(original_labels_train_ann_bm)
original_labels_train_ann_bm['original_index'] = y_train.index
original_labels_train_ann_bm['pred_ANN_BM'] = original_labels_train_ann_bm[0]
original_labels_train_ann_bm = original_labels_train_ann_bm.drop(columns=0)

In [ ]:
joined_set_ann_bmm = original_labels_train_ann_bm.merge(original_labels_test_ann_BM, on='original_index', how='outer')
joined_set_ann_bmm['pred_ANN_BM'] = joined_set_ann_bmm['pred_ANN_BM_x'].combine_first(joined_set_ann_bmm['pred_ANN_BM_y'])
joined_set_ann_bmm.drop(['pred_ANN_BM_x', 'pred_ANN_BM_y'], axis=1, inplace=True)
joined_set_ann_bmm

In [ ]:

Original_frame = Original_frame.merge(joined_set_ann_bmm, on= 'original_index', how='outer')
Original_frame

# predict with best model CNN 

In [ ]:
# define the function to build the model

from kerastuner import RandomSearch

def build_model(hp):
    model_cnn_bm = Sequential()
    model_cnn_bm.add(Conv1D(hp.Int('conv1_units', min_value=32, max_value=128, step=32),  kernel_size=hp.Choice('conv1_kernel', values=[3,5]), activation='relu', input_shape=(818, 1)))
    model_cnn_bm.add(MaxPooling1D(pool_size=hp.Choice('max_pool1', values=[2,4])))
    model_cnn_bm.add(Dropout(hp.Float('dropout1', min_value=0.2, max_value=0.5, step=0.1)))
    model_cnn_bm.add(Flatten())
    model_cnn_bm.add(Dense(hp.Int('dense1_units', min_value=32, max_value=128, step=32), activation='relu'))
    model_cnn_bm.add(Dropout(hp.Float('dropout2', min_value=0.2, max_value=0.5, step=0.1)))
    model_cnn_bm.add(Dense(17, activation='softmax'))
    model_cnn_bm.compile(loss='categorical_crossentropy', optimizer=Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')), metrics=['Recall','Precision', 'accuracy'])
    return model_cnn_bm

# define the tuner
tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=5,executions_per_trial=1,directory='tuner_results_migrated',project_name='my_cnn_tuner_join_test_to_dipl')

# load your data
# X_train, y_train, X_test, y_test = ...

# train the tuner
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# get the best model
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model_cnn = tuner.hypermodel.build(best_hyperparameters)

# evaluate the best model on the test data
loss, recall, precision, accuracy = best_model_cnn.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Recall:', recall)
print('Test Precision:', precision)
print('Test Accuracy:', accuracy)
print('Best Hyperparameters:', best_hyperparameters)

best_model_cnn.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
import matplotlib.pyplot as plt

# Extract the training and validation accuracy
acc = best_model_cnn.history.history['accuracy']
val_acc = best_model_cnn.history.history['val_accuracy']
val_prec = best_model_cnn.history.history['val_precision']
prec = best_model_cnn.history.history['precision']
val_recall = best_model_cnn.history.history['val_recall']
recall = best_model_cnn.history.history['recall']

# Plot the accuracy
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Accuracy of Best Model CNN')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from keras.utils import to_categorical
import numpy as np

# Generate predictions on the test set
y_bm_pred_test_cnn = best_model_cnn.predict(X_test)

pred_classes = np.argmax(y_bm_pred_test_cnn, axis=1)
true_classes = np.argmax(np.array(y_test), axis=1)


conf_matrix = confusion_matrix(true_classes, pred_classes, labels=range(17))

plt.figure(figsize =(10, 5)) 
sns.heatmap(conf_matrix, annot = True, fmt ="d")

In [ ]:
from keras.callbacks import TensorBoard

tensorboard_callback = TensorBoard(log_dir='./logsCNN', histogram_freq=1)

best_model_cnn.fit(X_train, y_train, epochs=50, callbacks=[tensorboard_callback])



In [ ]:
conf_matrix

In [ ]:
%tensorboard --logdir=./logsCNN


In [ ]:
y_pred_cnn_bm_test = best_model_cnn.predict(X_test)
label_mapping_cnn_bm = {index: label for index, label in enumerate(encoderY.get_feature_names_out())}
integer_labels_cnn_bm = np.argmax(y_pred_cnn_bm_test, axis=1)
original_labels_test_cnn_bm = [label_mapping_cnn_bm[x] for x in integer_labels_cnn_bm]
original_labels_test_cnn_bm = pd.DataFrame(original_labels_test_cnn_bm)
original_labels_test_cnn_bm['original_index'] = y_test.index
original_labels_test_cnn_bm['pred_CNN_BM'] = original_labels_test_cnn_bm[0]
original_labels_test_cnn_bm = original_labels_test_cnn_bm.drop(columns=0)
#prediction on train set 
#label mapping for training predictions
y_pred_train_cnn_bm = best_model_cnn.predict(X_train)
label_mapping_train_cnn_bm = {index: label for index, label in enumerate(encoderY.get_feature_names_out())}
integer_labels_train_bm_cnn = np.argmax(y_pred_train_cnn_bm, axis=1)
original_labels_train_bm_cnn = [label_mapping_train_cnn_bm[x] for x in integer_labels_train_bm_cnn]
original_labels_train_bm_cnn = pd.DataFrame(original_labels_train_bm_cnn)
original_labels_train_bm_cnn['original_index'] = y_train.index
original_labels_train_bm_cnn['pred_CNN_BM'] = original_labels_train_bm_cnn[0]
original_labels_train_bm_cnn = original_labels_train_bm_cnn.drop(columns=0)
joined_set_cnn_bm = original_labels_train_bm_cnn.merge(original_labels_test_cnn_bm, on='original_index', how='outer')
joined_set_cnn_bm['pred_CNN_BM'] = joined_set_cnn_bm['pred_CNN_BM_x'].combine_first(joined_set_cnn_bm['pred_CNN_BM_y'])
joined_set_cnn_bm.drop(['pred_CNN_BM_x', 'pred_CNN_BM_y'], axis=1, inplace=True)
joined_set_cnn_bm

In [ ]:
Original_frame = Original_frame.merge(joined_set_cnn_bm, on= 'original_index', how='outer')
Original_frame

Original_frame.to_csv('updated_results.csv')